## Making Choropleth's for FAO Land use data

### 1) Additional packages to import are geopandas and descartes. 

In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import descartes
pd.set_option('display.max_rows',500, 'display.max_columns', None)
%matplotlib inline

### 2) Import world map shapefile

In [ ]:
shapefile = 'ne_110m_admin_0_countries_lakes-Copy1.shp'

In [ ]:
world_gdf = gpd.read_file(shapefile)[['ADMIN','NAME','geometry']]
world_gdf.head()

In [ ]:
world_gdf.columns = ['Country','Country_short','geometry']
world_gdf.head(2)

In [ ]:
world_gdf.plot()
plt.show()

In [ ]:
world_gdf.info()

#### 2.1) Remove Antarctica

In [ ]:
print(world_gdf[world_gdf['Country']=='Antarctica'])

In [ ]:
world_gdf = world_gdf.drop(world_gdf.index[159])

In [ ]:
world_gdf.plot();

### 3) Import countries_land_use.csv file saved from the Jupyter notebook EDA of Land use FAO data_blog

In [ ]:
countries_land_use = pd.read_csv('countries_land_use.csv')

In [ ]:
countries_land_use.head(3)

#### 3.1) Rename countries to world_gdf names

In [ ]:
countries_land_use['Country'].unique()

In [ ]:
world_country_names = world_gdf['Country'].to_list()

In [ ]:
world_country_names.sort()
world_country_names

In [ ]:
rename_map = {'Bolivia (Plurinational State of)':'Bolivia','Brunei Darussalam':'Brunei',
              'Falkland Islands (Malvinas)':'Falkland Islands','Iran (Islamic Republic of)':'Iran',
              'Lao People''s Democratic Republic':'Laos','Democratic People''s Republic of Korea':'North Korea',
              'Republic of Korea':'South Korea','Serbia':'Republic of Serbia','Sudan(former)':'Sudan',
              'Syrian Arab Republic':'Syria','Timor-Leste':'East Timor','USSR':'Russia','Russian Federation':'Russia',
              'Venezuela (Bolivarian Republic of)':'Venezuela','Viet Nam':'Vietnam'}

In [ ]:
countries_land_use['Country']=countries_land_use['Country'].map(rename_map).fillna(countries_land_use['Country'])

#### 3.2) Create subset of DataFrame for arable land use

In [ ]:
arable = countries_land_use[countries_land_use['Land_use']=='Arable land']

In [ ]:
Rus = (arable[arable['Country']== 'Russia'])
Rus

In [ ]:
Rus_sum = Rus.sum(skipna=True)
Rus_DF = pd.DataFrame(Rus_sum)
Rus_DF = Rus_DF.T
Rus_DF['Country'] = Rus_DF['Country'].replace({'RussiaRussia':'Russia'})
Rus_DF['Land_use'] = Rus_DF['Land_use'].replace({'Arable landArable land': 'Arable land'})
Rus_DF['Element'] = Rus_DF['Element'].replace({'AreaArea':'Area'})
Rus_DF

In [ ]:
arable = arable.drop([3901,4962])

In [ ]:
arable = arable.append(Rus_DF, ignore_index=True)
arable.tail()

In [ ]:
arable['Country'].value_counts()

### 4) Merge world GeoDataFrame and Arable DataFrame
Create a GeoDataFrame that contains both polygon geometry and land use data

In [ ]:
arable_gdf = world_gdf.merge(arable, on='Country', how='outer')

In [ ]:
type(arable_gdf)

In [ ]:
arable_gdf

#### 4.1) Fill NaN with zero
Country rows with NaN will not appear on the map

In [ ]:
arable_gdf.loc[:,'1961':'2017'] = arable_gdf.loc[:,'1961':'2017'].fillna(0)

In [ ]:
arable_gdf

### 5) Plot Choropleth

In [ ]:
plt.rcParams['figure.figsize'] = [20, 10]
arable_gdf.plot(column = '2017', cmap = 'Oranges', edgecolor = 'gray', legend=True)
plt.show()

#### 5.1) Customise the plot and colorbar
To customize the plot and colorbar define the plot axes as ax and the legend axes as cax, then pass these to the .plot() function. The below example uses the mpl_toolkits make_axes_locatable function to vertically align the plot and legend axes. 

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, 1, figsize=(20,10))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

arable_gdf.plot(column = '2017', ax=ax, legend=True, cax=cax, cmap = 'Oranges', edgecolor = 'gray', linewidth=0.5)

ax.set_title('Hectares used for arable land, 2017', fontsize=20)
cax.set_title('Hectares', fontsize=14);

In [ ]:
#plt.rcParams(['figure.figsize'] = [90, 90], 'fontsize' = 20)

### Repeat for Permanent Meadows and Pastures

In [ ]:
perm_meadows = countries_land_use[countries_land_use['Land_use']=='Land under perm. meadows and pastures']

In [ ]:
Rus2 = (perm_meadows[perm_meadows['Country']== 'Russia'])
Rus2

In [ ]:
Rus2_sum = Rus2.sum(skipna=True)
Rus2_DF = pd.DataFrame(Rus2_sum)
Rus2_DF = Rus2_DF.T
Rus2_DF['Country'] = Rus2_DF['Country'].replace({'RussiaRussia':'Russia'})
Rus2_DF['Land_use'] = Rus2_DF['Land_use'].replace({'Land under perm. meadows and pasturesLand under perm. meadows and pastures': 'Land under perm. meadows and pastures'})
Rus2_DF['Element'] = Rus2_DF['Element'].replace({'AreaArea':'Area'})
Rus2_DF

In [ ]:
perm_meadows = perm_meadows.drop([3904,4964])
perm_meadows = perm_meadows.append(Rus2_DF, ignore_index=True)
perm_meadows.tail()

In [ ]:
perm_meadows['Country'].value_counts()

In [ ]:
perm_meadows_gdf = world_gdf.merge(perm_meadows, on='Country', how='outer')

In [ ]:
type(perm_meadows_gdf)

In [ ]:
perm_meadows_gdf

In [ ]:
perm_meadows_gdf.loc[:,'1961':'2017'] = perm_meadows_gdf.loc[:,'1961':'2017'].fillna(0)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, 1, figsize=(20,10))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

perm_meadows_gdf.plot(column = '2017', ax=ax, legend=True, cax=cax, cmap = 'Greens', edgecolor = 'gray', linewidth=0.5)

ax.set_title('Hectares used for permanent meadows & pasture land, 2017', fontsize=20)
cax.set_title('Hectares', fontsize=14);

plt.show()

In [ ]:
#plt.rcParams(['figure.figsize'] = [90, 90], 'fontsize' = 20)

In [ ]:
fig, ax = plt.subplots(1,1)

perm_meadows_gdf.plot(column = '2017', ax=ax, legend=True, 
                      legend_kwds={'pad': 0.02,
                                   'label':"Permanent meadows & pastures (ha)",
                                   'orientation':"horizontal"},
                      cmap = 'Greens', edgecolor = 'gray')

#plt.xticks(fontsize=200)


### 6) Choropleth showing change in hectares in use since 1961

Make a copy of arable_gdf as I don't want to change the original.

In [ ]:
arable_gdf_diff = arable_gdf

Create a column to contain the values for the calculation 2017 hectares - 1961 hectares, to see how hectares in use have changed since 1961.

In [ ]:
arable_gdf_diff['2017-1961'] = arable_gdf_diff['2017'] - arable_gdf_diff['1961']

In [ ]:
arable_gdf_diff

#### 6.1) Normalise the colorbar
By default any colorbar applied in matplotlib will diverge from the midpoint between the max and min values of the plotted column e.g. 1000 to -4500. This is not very useful when using divering colormaps to show positive and negative values as the plot below shows. The zero point is represented by blue and some negative values are also blue, when what we want is the colourmap to diverge from zero, the grey midpoint color, positive values to be blue and negative values to be red.  

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as colors

fig, ax = plt.subplots(1, 1, figsize=(20,10))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

arable_gdf_diff.plot(column = '2017-1961', ax=ax, legend=True, cax=cax, cmap = 'coolwarm_r', edgecolor = 'gray', linewidth=0.5)
# _r reverses the colormap so red represents negative values

ax.set_title('Change in hectares used for Arable land between 1961 and 2017', fontsize=20)
cax.set_title('Hectares', fontsize=14);

#### 6.2) Rest color midpoint to zero, DivergingNorm Function
Normalise the colormap around the zero centerpoint by using the DivergingNorm function in Matplotlib as shown below. The resulting choropleth is a much clearer representation of how land use has changed between 1961 and 2017.

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.colors as colors

fig, ax = plt.subplots(1, 1, figsize=(20,10))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

#normalise the colormap around zero
vmin, vmax, vcenter = arable_gdf_diff['2017-1961'].min(), arable_gdf_diff['2017-1961'].max(), 0
divnorm = colors.DivergingNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)


arable_gdf_diff.plot(column = '2017-1961', ax=ax, legend=True, cax=cax, cmap = 'coolwarm_r', norm=divnorm, edgecolor = 'gray', 
                     linewidth=0.5)
# _r reverses the colormap so red represents negative values

ax.set_title('Change in hectares used for Arable land between 1961 and 2017', fontsize=20)
cax.set_title('Hectares', fontsize=14);

In [ ]:
perm_meadows_gdf_diff = perm_meadows_gdf
perm_meadows_gdf_diff['2017-1961'] = perm_meadows_gdf_diff['2017'] - perm_meadows_gdf_diff['1961']
perm_meadows_gdf_diff

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

fig, ax = plt.subplots(1, 1, figsize=(20,10))

divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.1)

vmin, vmax, vcenter = perm_meadows_gdf_diff['2017-1961'].min(), perm_meadows_gdf_diff['2017-1961'].max(), 0
divnorm = colors.DivergingNorm(vmin=vmin, vcenter=vcenter, vmax=vmax)

perm_meadows_gdf_diff.plot(column = '2017-1961', ax=ax, legend=True, cax=cax, cmap = 'coolwarm_r', norm=divnorm, edgecolor = 'gray', linewidth=0.5)

ax.set_title('Change in hectares used for Permanent meadows & pasture land between 1961 and 2017', fontsize=20)
cax.set_title('Hectares', fontsize=14);